In [ ]:
import numpy as np
import pandas as pd

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Intel SIP/Reviews.csv',encoding = 'unicode_escape') #Loading data
data.drop_duplicates(keep='first')
df = data[['review_id','text','title']]
df = df.rename(columns={'review_id':'id', 'text':'document', 'title':'summary'})
df = df.dropna()
df['document']= df['document'].apply(lambda x: x.lower())
df['summary'] = df['summary'].apply(lambda x: x.lower())


In [ ]:
!pip install datasets transformers rouge-score nltk -q
!pip install -U accelerate
!pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 57.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 54.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 6.2 MB/s eta 0:00:00


In [ ]:
import torch
import datasets
from datasets import Dataset
from datasets import load_metric
from transformers import AutoTokenizer

model_checkpoint ='t5-small'

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

pad_on_right = tokenizer.padding_side == "right"
max_input_length = 1024
max_target_length = 64
df = df.sample(frac=1).reset_index(drop=True)
train = df[:3350]
valid = df[3350:3690]
test = df[-10:]
def preprocess_function(examples):
    inputs = ['summarize:' + doc for doc in examples["document"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True,padding='max_length')

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
import torch
import datasets
from datasets import Dataset

train = Dataset.from_pandas(train)
valid = Dataset.from_pandas(valid)
tokenized_train = train.map(preprocess_function, batched=True) # Tokeninzing the data
tokenized_valid = valid.map(preprocess_function, batched=True)
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

Map:   0%|          | 0/3350 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3864: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/340 [00:00<?, ? examples/s]

##Fine Tuning

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [ ]:
batch_size = 16
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
import nltk
import numpy as np
# Data cleaning
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
import gc
gc.collect()

90

In [ ]:
torch.cuda.empty_cache()
%env WANDB_DISABLED=True
model_name = model_checkpoint.split("/")[-1]
# model arguments
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


env: WANDB_DISABLED=True


In [ ]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_valid,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
metric = load_metric("rouge")
trainer.train() # Training model

<ipython-input-21-f113b7554d0b>:10: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("rouge")


You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,2.236196,29.308200,10.878600,25.948500,26.092800,18.614700
2,No log,2.140385,30.205600,11.404800,26.723800,26.737000,16.452900
3,2.522700,2.130263,30.416700,11.599800,27.092400,27.080100,16.391200


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


TrainOutput(global_step=630, training_loss=2.486493355887277, metrics={'train_runtime': 555.1835, 'train_samples_per_second': 18.102, 'train_steps_per_second': 1.135, 'total_flos': 2720370209587200.0, 'train_loss': 2.486493355887277, 'epoch': 3.0})

In [ ]:
trainer.save_model('/content/drive/MyDrive/Intel SIP/my_model.keras')

In [ ]:
#Testing the model
eval_dataset = Dataset.from_pandas(test)
eval_dataset = eval_dataset.map(
                preprocess_function,
                batched=True)
predict_results = trainer.predict(
            eval_dataset,max_length=128, num_beams=3)

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [ ]:
metrics = predict_results.metrics
metrics

{'test_loss': 2.2563092708587646,
 'test_rouge1': 29.4961,
 'test_rouge2': 12.3598,
 'test_rougeL': 27.3966,
 'test_rougeLsum': 27.3818,
 'test_gen_len': 27.0,
 'test_runtime': 2.289,
 'test_samples_per_second': 4.369,
 'test_steps_per_second': 0.437}

## Output of text summarization

In [57]:
input = """About this item:
 1 Dynamic Driver, HIFI Bass Sound KZ EDX Pro is Equipped with one new 10mm Dual magnetic circuit dynamic driver unit, Tuning focuses on the popular directions. The strong magnetic force enhances the low-frequency density. Restore sound details, bass bursts are more powerful. Rich texture and full of powerful bass. Bring you a new kind of auditory sense.
 Appearance Upgraded: Simple Stylish Design Resin inlaid metal technology, Highlight the extraordinary texture. The combination of resin and the wide range of color options display modern and eye-catching, fashionable earphones. Bright streamer, adds luster texture, full of beauty. Suitable for stage, outing, show, and other scenes.
 Cable Upgraded: 5N OFC Oxygen-free Copper Wire The Unit owns low impedance and high sensitivity, mobile phones can also be easily pushed directly. Double Parallel wires can effectively prevent winding and have a longer service life. Standard 0.75mm 2Pins gold-plated pins, exchangeable 2PIN wire design improves the playability. Search Yinyoo earphones cable, discover more HIFI cables.
 Comfortable to Wear As always, Ergonomic design, lightweight cavity and soft fit the auricle, and new PU memory ear hook, which can make you comfortable and stable to wear. Effectively more reduce external noise interference after being worn tightly, then you can just enjoy music by adjusting low volume.
 What You Will Get? Package includes 1 pair headset, 1* earphones cable, 3 pairs of ear tips, and a user manual. you will get the one-year protective time and friendly customer service from the date of purchase. If there are any problems, pls feel free to contact customer service to help you. Yinyoo Audio has Formal authorization from KZ company, all products are original.
"""
out = {'id':[1],'document':[input],'summary':[""]}
out = pd.DataFrame.from_dict(out)
eval_dataset = Dataset.from_pandas(out)
eval_dataset = eval_dataset.map(
                preprocess_function,
                batched=True)
predict_results = trainer.predict(
            eval_dataset,max_length=256, num_beams=3)
if args.predict_with_generate:
    predictions = tokenizer.batch_decode(predict_results.predictions, skip_special_tokens=True, clean_up_tokenization_spaces=True)
    predictions = [pred.strip() for pred in predictions]


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3864: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [56]:
print("Summary is : "+predictions[0])

Summary is : HIFI Bass Sound KZ EDX Pro is Equipped with one new 10mm dual magnetic circuit dynamic driver unit, Tuning focuses on the popular directions
